### Load the Implant Data

In [3]:
import pandas as pd
import openpyxl
import re

# Load the Excel file
excel_file = 'Implant-Data.xlsx'

# Get all sheet names
workbook = openpyxl.load_workbook(excel_file)
sheet_names = workbook.sheetnames

# Create a dictionary to store dataframes
dataframes = {}

def clean_sheet_name(name):
    # Replace spaces and special characters with underscores
    # Remove any characters that aren't alphanumeric or underscore
    cleaned_name = re.sub(r'\W+', '_', name)
    # Ensure the name starts with a letter or underscore
    if not cleaned_name[0].isalpha() and cleaned_name[0] != '_':
        cleaned_name = '_' + cleaned_name
    return cleaned_name

# Load each sheet into a separate dataframe
for sheet in sheet_names:
    # Read the sheet, converting all columns to string (object) dtype
    df = pd.read_excel(excel_file, sheet_name=sheet, dtype=str)
    
    # Create a valid variable name for the dataframe
    df_name = f"df_{clean_sheet_name(sheet)}"
    
    # Store the dataframe in the dictionary
    dataframes[df_name] = df
    
    # Create a global variable with the dataframe name
    globals()[df_name] = df

print("Dataframes created:")
for df_name, original_sheet in zip(dataframes.keys(), sheet_names):
    print(f"- {df_name} (original sheet name: '{original_sheet}')")

Dataframes created:
- df_CERNER (original sheet name: 'CERNER')
- df_Prescribed_List (original sheet name: 'Prescribed List')


In [4]:
import pandas as pd

# Assuming df_CERNER and df_Prescribed_List are already loaded

# Convert the relevant columns to sets for efficient comparison
cerner_codes = set(df_CERNER['SOURCE_IDENTIFIER'])
prescribed_codes = set(df_Prescribed_List['Billing Code'])

# Find new codes (in Prescribed List but not in CERNER)
new_codes = list(prescribed_codes - cerner_codes)

# Find unfound codes (in CERNER but not in Prescribed List)
unfound_codes = list(cerner_codes - prescribed_codes)

# Sort the lists for easier reading
new_codes.sort()
unfound_codes.sort()

# Print the results
print(f"Number of new codes: {len(new_codes)}")
print(f"Number of unfound codes: {len(unfound_codes)}")

# Optional: Print the first few items of each list
print("\nFirst 5 new codes:")
print(new_codes[:5])
print("\nFirst 5 unfound codes:")
print(unfound_codes[:5])

# Optional: Save the lists to CSV files
pd.Series(new_codes).to_csv('new_codes.csv', index=False, header=['New Codes'])
pd.Series(unfound_codes).to_csv('unfound_codes.csv', index=False, header=['Unfound Codes'])

Number of new codes: 878
Number of unfound codes: 2419

First 5 new codes:
['AK013', 'AL059', 'AL060', 'AO049', 'AO050']

First 5 unfound codes:
['AA009', 'AAA02', 'AAA05', 'AAA06', 'AAA08']


In [14]:
# Count distinct codes
distinct_cerner_count = df_CERNER['SOURCE_IDENTIFIER'].nunique()
distinct_gov_count = df_Prescribed_List['Billing Code'].nunique()
# Print the result
print(f"The number of distinct codes in Oracle (Cerner) SOURCE_IDENTIFIER is: {distinct_cerner_count}")
print(f"The number of distinct codes in The Government Data 'Billing Code' is: {distinct_gov_count}")

print("New Codes to be added:")
print(len(new_codes))
print("Unfound Codes to be deactivated:")
print(len(unfound_codes))

The number of distinct codes in Oracle (Cerner) SOURCE_IDENTIFIER is: 11214
The number of distinct codes in The Government Data 'Billing Code' is: 9673
New Codes to be added:
878
Unfound Codes to be deactivated:
2419
